# Reconhecimento Óptico de Caracteres (OCR) com Agrupamento de Contornos

Este notebook demonstra um fluxo de trabalho de OCR aprimorado, focado em lidar com caracteres que são compostos por vários contornos separados (por exemplo, 'i', 'j', '=', ou caracteres acentuados). A abordagem principal é agrupar contornos que estão próximos horizontalmente antes de realizar o reconhecimento.

O processo inclui:
1.  **Pré-processamento de Imagem**: Binarização da imagem de entrada para isolar o texto.
2.  **Análise de Espaçamento**: Análise estatística dos espaços entre os contornos para sugerir um limiar ideal (`max_gap`) para o agrupamento.
3.  **Segmentação com Agrupamento**: Identificação e agrupamento de contornos próximos para formar símbolos completos.
4.  **Previsão**: Uso de um modelo CNN pré-treinado para classificar cada símbolo agrupado.
5.  **Visualização**: Exibição dos resultados, destacando os símbolos compostos por múltiplos componentes.

Esta técnica melhora a precisão do OCR para fontes e idiomas onde os caracteres não são sempre um único componente conectado.

In [1]:
import numpy as np
import tensorflow as tf
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import json
from pathlib import Path
import cv2

### Localização da Raiz do Projeto

A função `find_project_root` sobe na árvore de diretórios a partir do local atual para encontrar um arquivo marcador (como `pyproject.toml` ou `.git`), garantindo que os caminhos para os dados e modelos sejam sempre relativos à raiz do projeto, independentemente de onde o notebook é executado.

In [2]:
def find_project_root(markers=("pyproject.toml", ".git")) -> Path:
    """Localiza o diretório raiz do projeto.

    Esta função sobe na árvore de diretórios a partir do diretório de trabalho atual
    até encontrar um arquivo ou pasta que sirva como marcador da raiz do projeto (por exemplo, 
    `.git` ou `pyproject.toml`). Isso garante que os caminhos para os arquivos de dados sejam 
    resolvidos corretamente, não importa de onde o script seja executado.

    Args:
        markers (tuple, optional): Uma tupla de nomes de arquivos/pastas que indicam a raiz.
                                     O padrão é ("pyproject.toml", ".git").

    Returns:
        Path: O caminho absoluto para o diretório raiz do projeto.
    """
    p = Path.cwd().resolve()
    for parent in [p, *p.parents]:
        if any((parent / m).exists() for m in markers):
            return parent
    return p

ROOT = find_project_root()
print(f"Raiz do projeto: {ROOT}")

### Configuração de Caminhos e Parâmetros

Define todos os caminhos de arquivo necessários, como o local do modelo, a imagem de teste e o arquivo de nomes de classes. Também define as dimensões de imagem (`IMG_HEIGHT`, `IMG_WIDTH`) que o modelo espera.

In [3]:
# Configurações
DATA_DIR = ROOT / "data"
MODEL_PATH = DATA_DIR / "processed" / "modelo_ocr_simbolos.keras"
IMAGE_PATH = DATA_DIR / "raw" / "ocr_test_image.png"
CLASS_NAMES_PATH = DATA_DIR / "raw" / "class_names.json"

IMG_HEIGHT = 64
IMG_WIDTH = 64

### Carregamento do Modelo e Nomes das Classes

Carrega o modelo de OCR Keras pré-treinado e o arquivo JSON que mapeia os índices de saída do modelo para os nomes de classes (caracteres Unicode).

In [4]:
# Carrega o modelo treinado
print("Carregando modelo...")
model = tf.keras.models.load_model(MODEL_PATH)
print("✓ Modelo carregado com sucesso!")
model.summary()

In [5]:
# Carrega os nomes das classes
with open(CLASS_NAMES_PATH, 'r', encoding='utf-8') as f:
    class_names = json.load(f)
print(f"✓ {len(class_names)} classes carregadas")
print(f"Exemplos: {class_names[:5]}")

### Função Utilitária para Conversão de Unicode

A função `unicode_to_char` converte uma representação de string de um ponto de código Unicode (por exemplo, `'U+0041'`) no caractere correspondente (por exemplo, `'A'`).

In [6]:
def unicode_to_char(unicode_str: str) -> str:
    """Converte uma string de ponto de código Unicode para um caractere.

    Lida com o formato 'U+XXXX' e o converte no caractere de texto correspondente.

    Args:
        unicode_str (str): A string representando o ponto de código (ex: 'U+0041').

    Returns:
        str: O caractere correspondente (ex: 'A') ou '?' se ocorrer um erro.
    """
    try:
        codepoint = int(unicode_str.replace('U+', ''), 16)
        return chr(codepoint)
    except:
        return '?'

### Carregamento e Pré-processamento da Imagem

A imagem de teste é carregada e exibida. Em seguida, a função `preprocess_image_for_ocr` a converte para escala de cinza, aplica um limiar adaptativo para binarizá-la e inverte as cores para garantir um fundo branco e texto preto, que é o formato esperado para a detecção de contornos.

In [7]:
# Carrega e exibe a imagem original
print(f"Carregando imagem: {IMAGE_PATH}")
original_image = Image.open(IMAGE_PATH)

plt.figure(figsize=(12, 4))
plt.imshow(original_image, cmap='gray')
plt.title('Imagem Original para OCR')
plt.axis('off')
plt.show()

print(f"Dimensões: {original_image.size}")
print(f"Modo: {original_image.mode}")

In [8]:
def preprocess_image_for_ocr(image: Image.Image) -> np.ndarray:
    """Pré-processa a imagem para otimizar a detecção de caracteres.

    Esta função executa as seguintes etapas:
    1. Converte a imagem para escala de cinza.
    2. Aplica um limiar gaussiano adaptativo para criar uma imagem binarizada clara.
    3. Garante que a imagem tenha um fundo branco e texto preto, invertendo se necessário,
       que é o formato ideal para a detecção de contornos do OpenCV.

    Args:
        image (Image.Image): O objeto de imagem Pillow de entrada.

    Returns:
        np.ndarray: A imagem pré-processada como um array NumPy.
    """
    # Converte para escala de cinza
    if image.mode != 'L':
        image = image.convert('L')

    # Converte para numpy array
    img_array = np.array(image)

    # Aplica threshold adaptativo para melhor binarização
    img_binary = cv2.adaptiveThreshold(
        img_array, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 11, 2
    )

    # Verifica se precisa inverter (modelo espera fundo branco, texto preto)
    # Se a média é baixa, a imagem está com fundo escuro
    if np.mean(img_binary) < 127:
        img_binary = 255 - img_binary

    return img_binary

In [9]:
# Pré-processa a imagem
print("Pré-processando imagem...")
processed_image = preprocess_image_for_ocr(original_image)

plt.figure(figsize=(12, 4))
plt.imshow(processed_image, cmap='gray')
plt.title('Imagem Pré-processada (Binarizada)')
plt.axis('off')
plt.show()

### Análise e Agrupamento de Contornos

Esta é a etapa central do notebook. Primeiro, `analyze_spacing` examina a imagem para encontrar os espaços horizontais entre todos os contornos detectados e sugere um limiar `max_gap`. Em seguida, `segment_characters_with_grouping` usa esse limiar para agrupar contornos próximos em um único "símbolo". Isso é crucial para reconhecer corretamente caracteres como "Ì" ou "k", que são compostos por partes separadas.

In [10]:
def segment_characters_with_grouping(image_array: np.ndarray,
                                     max_gap: int = 15,
                                     min_component_area: int = 10) -> list:
    """Segmenta caracteres, agrupando múltiplos contornos próximos em um único símbolo.

    Esta função é projetada para lidar com caracteres compostos por partes desconectadas
    (como 'i', 'j', ou caracteres com acentos). Ela primeiro encontra todos os contornos,
    depois os agrupa horizontalmente se a lacuna (gap) entre eles for menor que `max_gap`.
    Finalmente, cria uma única caixa delimitadora e imagem para cada grupo.

    Args:
        image_array (np.ndarray): A imagem binarizada (fundo branco).
        max_gap (int, optional): A distância horizontal máxima entre as caixas delimitadoras
                                 para considerá-las parte do mesmo símbolo. O padrão é 15.
        min_component_area (int, optional): A área mínima de um contorno para ser considerado
                                            um componente válido, para filtrar ruídos. O padrão é 10.

    Returns:
        list: Uma lista de dicionários, onde cada dicionário representa um símbolo segmentado
              e contém 'bbox', 'image', 'components', e 'component_boxes'.
    """
    # 1. Encontra todos os contornos
    contours, _ = cv2.findContours(
        255 - image_array,
        cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE
    )

    # 2. Extrai bounding boxes e filtra ruído
    boxes = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        area = w * h
        if area >= min_component_area:
            boxes.append({
                'x': x, 'y': y, 'w': w, 'h': h,
                'x_center': x + w // 2,
                'x_end': x + w,
                'contour': contour
            })

    # 3. Ordena boxes da esquerda para direita
    boxes = sorted(boxes, key=lambda b: b['x'])

    if not boxes:
        return []

    # 4. Agrupa contornos próximos em símbolos compostos
    symbol_groups = []
    current_group = [boxes[0]]

    for i in range(1, len(boxes)):
        prev_box = current_group[-1]
        curr_box = boxes[i]

        # Calcula gap horizontal entre o fim do anterior e início do atual
        gap = curr_box['x'] - prev_box['x_end']

        # Se gap é pequeno, agrupa no mesmo símbolo
        if gap <= max_gap:
            current_group.append(curr_box)
        else:
            # Gap grande: novo símbolo
            symbol_groups.append(current_group)
            current_group = [curr_box]

    # Adiciona último grupo
    symbol_groups.append(current_group)

    # 5. Cria bounding box composto para cada grupo
    segmented_symbols = []

    for group in symbol_groups:
        # Calcula bbox que engloba todos os componentes
        x_min = min(box['x'] for box in group)
        y_min = min(box['y'] for box in group)
        x_max = max(box['x'] + box['w'] for box in group)
        y_max = max(box['y'] + box['h'] for box in group)

        # Adiciona padding
        padding = 5
        x1 = max(0, x_min - padding)
        y1 = max(0, y_min - padding)
        x2 = min(image_array.shape[1], x_max + padding)
        y2 = min(image_array.shape[0], y_max + padding)

        # Extrai região da imagem
        symbol_img = image_array[y1:y2, x1:x2]

        # Informações do símbolo composto
        bbox = (x1, y1, x2 - x1, y2 - y1)
        component_count = len(group)

        segmented_symbols.append({
            'bbox': bbox,
            'image': symbol_img,
            'components': component_count,
            'component_boxes': [(b['x'], b['y'], b['w'], b['h']) for b in group]
        })

    return segmented_symbols


def analyze_spacing(image_array: np.ndarray) -> dict:
    """Analisa o espaçamento horizontal entre contornos para sugerir um `max_gap` ideal.

    Esta função calcula a distância horizontal (gap) entre as caixas delimitadoras de todos 
    os contornos adjacentes. Ela então retorna estatísticas sobre esses gaps, incluindo uma 
    sugestão para o parâmetro `max_gap` baseada no 25º percentil, que tende a capturar 
    espaços dentro de caracteres compostos.

    Args:
        image_array (np.ndarray): A imagem binarizada (fundo branco).

    Returns:
        dict: Um dicionário contendo a lista de gaps e estatísticas (média, mediana, etc.),
              incluindo um `suggested_max_gap`.
    """
    contours, _ = cv2.findContours(
        255 - image_array,
        cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE
    )

    boxes = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w * h > 10:
            boxes.append({'x': x, 'w': w, 'x_end': x + w})

    boxes = sorted(boxes, key=lambda b: b['x'])

    # Calcula gaps
    gaps = []
    for i in range(1, len(boxes)):
        gap = boxes[i]['x'] - boxes[i-1]['x_end']
        gaps.append(gap)

    if not gaps:
        return {'gaps': [], 'suggested_max_gap': 15}

    gaps = np.array(gaps)

    # Estatísticas
    stats = {
        'gaps': gaps.tolist(),
        'mean': float(np.mean(gaps)),
        'median': float(np.median(gaps)),
        'std': float(np.std(gaps)),
        'min': float(np.min(gaps)),
        'max': float(np.max(gaps)),
        'suggested_max_gap': int(np.percentile(gaps, 25))  # 25º percentil
    }

    return stats

In [11]:
# Célula para análise de espaçamento (execute primeiro para calibrar)
print("Analisando espaçamento entre contornos...")
spacing_stats = analyze_spacing(processed_image)

print(f"\nEstatísticas de gaps:")
print(f"  Média: {spacing_stats['mean']:.1f} pixels")
print(f"  Mediana: {spacing_stats['median']:.1f} pixels")
print(f"  Desvio padrão: {spacing_stats['std']:.1f} pixels")
print(f"  Min/Max: {spacing_stats['min']:.0f} / {spacing_stats['max']:.0f} pixels")
print(f"  ✓ max_gap sugerido: {spacing_stats['suggested_max_gap']} pixels")

# Visualiza distribuição de gaps
plt.figure(figsize=(10, 4))
plt.hist(spacing_stats['gaps'], bins=20, edgecolor='black')
plt.axvline(spacing_stats['suggested_max_gap'], color='r', linestyle='--',
            label=f"Sugestão: {spacing_stats['suggested_max_gap']}px")
plt.xlabel('Gap (pixels)')
plt.ylabel('Frequência')
plt.title('Distribuição de Espaçamento entre Contornos')
plt.legend()
plt.show()

In [12]:
# Célula para segmentação com agrupamento
print("\nSegmentando símbolos compostos...")

# Use o max_gap sugerido ou ajuste manualmente
MAX_GAP = spacing_stats['suggested_max_gap']  # ou um valor fixo como 15

segmented_symbols = segment_characters_with_grouping(
    processed_image,
    max_gap=MAX_GAP,
    min_component_area=10
)

print(f"✓ {len(segmented_symbols)} símbolos detectados")

# Visualiza símbolos com informação de componentes
if len(segmented_symbols) > 0:
    n_symbols = len(segmented_symbols)
    cols = min(10, n_symbols)
    rows = (n_symbols + cols - 1) // cols

    plt.figure(figsize=(15, 2 * rows))
    for i, symbol in enumerate(segmented_symbols):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(symbol['image'], cmap='gray')
        plt.title(f"#{i+1}\n{symbol['components']} comp.", fontsize=9)
        plt.axis('off')
    plt.suptitle(f'Símbolos Segmentados (max_gap={MAX_GAP}px)')
    plt.tight_layout()
    plt.show()

    # Estatísticas
    comp_counts = [s['components'] for s in segmented_symbols]
    print(f"\nComponentes por símbolo:")
    print(f"  Média: {np.mean(comp_counts):.1f}")
    print(f"  Min/Max: {min(comp_counts)} / {max(comp_counts)}")
else:
    print("⚠ Nenhum símbolo detectado!")

### Preparação de Símbolos para Previsão

Cada imagem de símbolo agrupado é processada pela função `prepare_char_for_prediction`. Ela centraliza o símbolo em uma tela quadrada, redimensiona-o para o tamanho de entrada do modelo e adiciona as dimensões de lote e canal necessárias.

In [13]:
def prepare_char_for_prediction(char_image: np.ndarray) -> np.ndarray:
    """Prepara a imagem de um caractere segmentado para a predição do modelo.

    Esta função pega a imagem de um único caractere, a centraliza em uma tela quadrada 
    (canvas) branca, redimensiona para as dimensões esperadas pelo modelo (64x64) e 
    remodela o array para o formato de entrada do modelo (1, 64, 64, 1).

    Args:
        char_image (np.ndarray): A imagem do caractere segmentado (fundo branco).

    Returns:
        np.ndarray: A imagem preparada, pronta para ser usada como entrada para o 
                    modelo de previsão.
    """
    # Cria canvas quadrado branco
    size = max(char_image.shape)
    canvas = np.ones((size, size), dtype=np.uint8) * 255

    # Centraliza o caractere
    y_offset = (size - char_image.shape[0]) // 2
    x_offset = (size - char_image.shape[1]) // 2
    canvas[y_offset:y_offset+char_image.shape[0],
           x_offset:x_offset+char_image.shape[1]] = char_image

    # Redimensiona para 64x64
    resized = cv2.resize(canvas, (IMG_WIDTH, IMG_HEIGHT),
                        interpolation=cv2.INTER_LANCZOS4)

    # Adiciona dimensões: (1, 64, 64, 1)
    # IMPORTANTE: NÃO normalizar aqui - o modelo tem Rescaling(1./255) interno
    prepared = resized.reshape(1, IMG_HEIGHT, IMG_WIDTH, 1)

    return prepared

### Execução do OCR e Exibição dos Resultados

O notebook itera sobre cada símbolo segmentado, o prepara para o modelo e executa a previsão. O caractere com a maior confiança é selecionado. O resultado final, incluindo o texto reconhecido, a confiança de cada previsão e o número de componentes, é impresso na tela.

In [ ]:
# Célula para OCR com os símbolos agrupados
print("\nRealizando OCR dos símbolos compostos...")
results = []

for i, symbol_data in enumerate(segmented_symbols):
    bbox = symbol_data['bbox']
    char_img = symbol_data['image']

    # Prepara para predição
    input_array = prepare_char_for_prediction(char_img)

    # Predição
    predictions = model.predict(input_array, verbose=0)
    predicted_idx = np.argmax(predictions[0])
    confidence = predictions[0][predicted_idx]

    # Converte para caractere
    unicode_class = class_names[predicted_idx]
    character = unicode_to_char(unicode_class)

    results.append({
        'index': i,
        'bbox': bbox,
        'unicode': unicode_class,
        'character': character,
        'confidence': confidence,
        'components': symbol_data['components'],
        'component_boxes': symbol_data['component_boxes'],
        'image': char_img
    })

    comp_info = f" ({symbol_data['components']} componentes)" if symbol_data['components'] > 1 else ""
    print(f"Símbolo {i+1}: '{character}' ({unicode_class}){comp_info} - Confiança: {confidence:.4f}")

recognized_text = ''.join([r['character'] for r in results])
print(f"\n{'='*60}")
print(f"TEXTO RECONHECIDO: {recognized_text}")
print(f"{'='*60}")
